## 1. Load library

In [69]:
# basic library
import pandas as pd
import random
import os
import numpy as np
import warnings
import time
import datetime

# option library
warnings.filterwarnings('ignore')

# preprocessing library
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

# model library
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
import lightgbm as lgb

# metrics library(평가지표)
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
from sklearn.metrics import classification_report

# validation library(검증)
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_validate

# optuna library
import optuna
from optuna.samplers import TPESampler
from sklearn.model_selection import train_test_split

In [3]:
# Seed 고정
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(37) 

## 2. Load Data

In [4]:
train_df = pd.read_csv('../train.csv')
test_df = pd.read_csv('../test.csv')

* Columns
 - Y_class : 제품 품질 상태(Target)
   + 0 : 적정 기준 미달 (부적합)
   + 1 : 적합
   + 2 : 적정 기준 초과 (부적합)
 - Y_Quality : 제품 품질 관련 정량적 수치
 - TIMESTAMP : 제품이 공정에 들어간 시각
 - LINE : 제품이 들어간 공정 LINE 종류  ('T050304','T050307','T100304','T010306','T010305')
 - PRODUCT_CODE : 제품의 CODE 번호('A_31', 'T_31', 'O_31')
 - X_1 ~ X_2875 : 공정 과정에서 추출되어 비식별화된 변수

## 3. 데이터 확인

In [5]:
# train데이터 
train_df

,PRODUCT_ID,Y_Class,Y_Quality,TIMESTAMP,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
0,TRAIN_000,1,0.533433,2022-06-13 5:14,T050304,A_31,NaN,NaN,NaN,NaN,...,39.34,40.89,32.56,34.09,77.77,NaN,NaN,NaN,NaN,NaN
1,TRAIN_001,2,0.541819,2022-06-13 5:22,T050307,A_31,NaN,NaN,NaN,NaN,...,38.89,42.82,43.92,35.34,72.55,NaN,NaN,NaN,NaN,NaN
2,TRAIN_002,1,0.531267,2022-06-13 5:30,T050304,A_31,NaN,NaN,NaN,NaN,...,39.19,36.65,42.47,36.53,78.35,NaN,NaN,NaN,NaN,NaN
3,TRAIN_003,2,0.537325,2022-06-13 5:39,T050307,A_31,NaN,NaN,NaN,NaN,...,37.74,39.17,52.17,30.58,71.78,NaN,NaN,NaN,NaN,NaN
4,TRAIN_004,1,0.531590,2022-06-13 5:47,T050304,A_31,NaN,NaN,NaN,NaN,...,38.70,41.89,46.93,33.09,76.97,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593,TRAIN_593,1,0.526546,2022-09-08 14:30,T100306,T_31,2.0,95.0,0.0,45.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
594,TRAIN_594,0,0.524022,2022-09-08 22:38,T050304,A_31,NaN,NaN,NaN,NaN,...,49.47,53.07,50.89,55.10,66.49,1.0,NaN,NaN,NaN,NaN
595,TRAIN_595,0,0.521289,2022-09-08 22:47,T050304,A_31,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
596,TRAIN_596,1,0.531375,2022-09-08 14:38,T100304,O_31,40.0,94.0,0.0,45.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
#test 데이터
test_df

,PRODUCT_ID,TIMESTAMP,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
0,TEST_000,2022-09-09 2:01,T100306,T_31,2.0,94.0,0.0,45.0,10.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TEST_001,2022-09-09 2:09,T100304,T_31,2.0,93.0,0.0,45.0,11.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,TEST_002,2022-09-09 8:42,T100304,T_31,2.0,95.0,0.0,45.0,11.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,TEST_003,2022-09-09 10:56,T010305,A_31,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,TEST_004,2022-09-09 11:04,T010306,A_31,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305,TEST_305,2022-11-05 11:18,T100306,T_31,2.0,91.0,0.0,45.0,10.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
306,TEST_306,2022-11-05 16:39,T100304,T_31,2.0,96.0,0.0,45.0,11.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
307,TEST_307,2022-11-05 16:47,T100306,T_31,2.0,91.0,0.0,45.0,10.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
308,TEST_308,2022-11-05 20:53,T100306,T_31,2.0,95.0,0.0,45.0,10.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


> #### 데이터 공정라인과 제품번호 확인

In [16]:
qual_col = ['LINE', 'PRODUCT_CODE']

for i in qual_col:
    print(np.unique(test_x[i]))

['T010305' 'T010306' 'T050304' 'T050307' 'T100304' 'T100306']
['A_31' 'O_31' 'T_31']


## 4. Data PreProcessing

In [48]:
# test data와 train data 에서 분류할 colums LINE,PRODUCT_CODE,X데이터만 남김
test_x = test_df.drop(columns = ['PRODUCT_ID','TIMESTAMP'])
train_x = train_df.drop(columns = ['PRODUCT_ID','TIMESTAMP','Y_Class','Y_Quality'])

In [49]:
# Train데이터에서 분류된 Y_Class train_y
train_y = train_df['Y_Class']

In [50]:
test_x

,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
0,T100306,T_31,2.0,94.0,0.0,45.0,10.0,0.0,51.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,T100304,T_31,2.0,93.0,0.0,45.0,11.0,0.0,45.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,T100304,T_31,2.0,95.0,0.0,45.0,11.0,0.0,45.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,T010305,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,T010306,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305,T100306,T_31,2.0,91.0,0.0,45.0,10.0,0.0,51.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
306,T100304,T_31,2.0,96.0,0.0,45.0,11.0,0.0,45.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
307,T100306,T_31,2.0,91.0,0.0,45.0,10.0,0.0,50.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
308,T100306,T_31,2.0,95.0,0.0,45.0,10.0,0.0,51.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
train_x

,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
0,T050304,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,39.34,40.89,32.56,34.09,77.77,NaN,NaN,NaN,NaN,NaN
1,T050307,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,38.89,42.82,43.92,35.34,72.55,NaN,NaN,NaN,NaN,NaN
2,T050304,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,39.19,36.65,42.47,36.53,78.35,NaN,NaN,NaN,NaN,NaN
3,T050307,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,37.74,39.17,52.17,30.58,71.78,NaN,NaN,NaN,NaN,NaN
4,T050304,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,38.70,41.89,46.93,33.09,76.97,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593,T100306,T_31,2.0,95.0,0.0,45.0,10.0,0.0,50.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
594,T050304,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,49.47,53.07,50.89,55.10,66.49,1.0,NaN,NaN,NaN,NaN
595,T050304,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
596,T100304,O_31,40.0,94.0,0.0,45.0,11.0,0.0,45.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
train_y

0      1
1      2
2      1
3      2
4      1
      ..
593    1
594    0
595    0
596    1
597    1
Name: Y_Class, Length: 598, dtype: int64

#### Label Encoding

In [37]:
qual_col = ['LINE','PRODUCT_CODE'] # 변환 할 피처

In [53]:
for i in qual_col:
    le = LabelEncoder()
    le = le.fit(train_x[i])
    train_x[i] = le.transform(train_x[i])
    
    for label in np.unique(test_x[i]): 
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test_x[i] = le.transform(test_x[i]) 

- qualitative to quantitative : 정성적 데이터(비정형데이터, 문자나 언어의 텍스트 파일)을 정량적 데이터로 수치화하는 전처리 작업
- le.fit() : 학습 시 고유 피처들을 학습하여 고유번호를 지정
- le.transform() : 각 피처값을 해당하는 학습된 고유값으로 변환
- .classes_ : 고유번호 0번부터 고유값에 대한 원본 데이터 값을 가지고 있음
- np.append(대상 어레이,추가할 값, 파라미터축)

In [54]:
train_x

,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
0,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,39.34,40.89,32.56,34.09,77.77,NaN,NaN,NaN,NaN,NaN
1,3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,38.89,42.82,43.92,35.34,72.55,NaN,NaN,NaN,NaN,NaN
2,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,39.19,36.65,42.47,36.53,78.35,NaN,NaN,NaN,NaN,NaN
3,3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,37.74,39.17,52.17,30.58,71.78,NaN,NaN,NaN,NaN,NaN
4,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,38.70,41.89,46.93,33.09,76.97,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593,5,2,2.0,95.0,0.0,45.0,10.0,0.0,50.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
594,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,49.47,53.07,50.89,55.10,66.49,1.0,NaN,NaN,NaN,NaN
595,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
596,4,1,40.0,94.0,0.0,45.0,11.0,0.0,45.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
test_x

,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
0,5,2,2.0,94.0,0.0,45.0,10.0,0.0,51.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4,2,2.0,93.0,0.0,45.0,11.0,0.0,45.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4,2,2.0,95.0,0.0,45.0,11.0,0.0,45.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305,5,2,2.0,91.0,0.0,45.0,10.0,0.0,51.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
306,4,2,2.0,96.0,0.0,45.0,11.0,0.0,45.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
307,5,2,2.0,91.0,0.0,45.0,10.0,0.0,50.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
308,5,2,2.0,95.0,0.0,45.0,10.0,0.0,51.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### 정규화

In [56]:
columns_x = train_x.columns.str.contains('X')
print(columns_x)

[False False  True ...  True  True  True]


In [57]:
x_col = train_x.columns[train_x.columns.str.contains('X')].tolist()
print(x_col[:10])

['X_1', 'X_2', 'X_3', 'X_4', 'X_5', 'X_6', 'X_7', 'X_8', 'X_9', 'X_10']


- .columns.str.contains("a") : a라는 문자가 들어있는 문자열 모두 찾음

In [58]:
train_x[x_col]

,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,X_10,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,39.34,40.89,32.56,34.09,77.77,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,38.89,42.82,43.92,35.34,72.55,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,39.19,36.65,42.47,36.53,78.35,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,37.74,39.17,52.17,30.58,71.78,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,38.70,41.89,46.93,33.09,76.97,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593,2.0,95.0,0.0,45.0,10.0,0.0,50.0,10.0,52.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
594,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,49.47,53.07,50.89,55.10,66.49,1.0,NaN,NaN,NaN,NaN
595,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
596,40.0,94.0,0.0,45.0,11.0,0.0,45.0,10.0,31.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
scaler = MinMaxScaler()
scaler.fit(train_x[x_col])
  
train_x[x_col] = scaler.transform(train_x[x_col])
test_x[x_col] = scaler.transform(test_x[x_col])
  
train_x.head()

,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
0,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.256757,0.248647,0.000000,0.122283,0.890487,NaN,NaN,NaN,NaN,NaN
1,3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.240754,0.300866,0.407899,0.164742,0.601770,NaN,NaN,NaN,NaN,NaN
2,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.251422,0.133929,0.355835,0.205163,0.922566,NaN,NaN,NaN,NaN,NaN
3,3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.199858,0.202110,0.704129,0.003057,0.559181,NaN,NaN,NaN,NaN,NaN
4,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.233997,0.275703,0.515978,0.088315,0.846239,NaN,NaN,NaN,NaN,NaN


#### 결측값(NaN) 평균으로 채우기

In [60]:
train_x = train_x.fillna(train_x.mean())# mean 연산시 nan값은 분석 대상에서 제외
test_x = test_x.fillna(train_x.mean())

In [61]:
train_x

,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
0,2,0,0.013821,0.541547,0.0,0.0,0.39255,0.0,0.223664,0.048711,...,0.256757,0.248647,0.000000,0.122283,0.890487,0.0,NaN,NaN,NaN,NaN
1,3,0,0.013821,0.541547,0.0,0.0,0.39255,0.0,0.223664,0.048711,...,0.240754,0.300866,0.407899,0.164742,0.601770,0.0,NaN,NaN,NaN,NaN
2,2,0,0.013821,0.541547,0.0,0.0,0.39255,0.0,0.223664,0.048711,...,0.251422,0.133929,0.355835,0.205163,0.922566,0.0,NaN,NaN,NaN,NaN
3,3,0,0.013821,0.541547,0.0,0.0,0.39255,0.0,0.223664,0.048711,...,0.199858,0.202110,0.704129,0.003057,0.559181,0.0,NaN,NaN,NaN,NaN
4,2,0,0.013821,0.541547,0.0,0.0,0.39255,0.0,0.223664,0.048711,...,0.233997,0.275703,0.515978,0.088315,0.846239,0.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593,5,2,0.009804,0.533333,0.0,0.0,0.00000,0.0,0.294118,0.000000,...,0.664555,0.592741,0.612072,0.719083,0.275426,0.0,NaN,NaN,NaN,NaN
594,2,0,0.013821,0.541547,0.0,0.0,0.39255,0.0,0.223664,0.048711,...,0.616999,0.578193,0.658169,0.835938,0.266593,0.0,NaN,NaN,NaN,NaN
595,2,0,0.013821,0.541547,0.0,0.0,0.39255,0.0,0.223664,0.048711,...,0.664555,0.592741,0.612072,0.719083,0.275426,0.0,NaN,NaN,NaN,NaN
596,4,1,0.382353,0.466667,0.0,0.0,1.00000,0.0,0.000000,0.000000,...,0.664555,0.592741,0.612072,0.719083,0.275426,0.0,NaN,NaN,NaN,NaN


#### 전체 결측값(Null)인 X변수 Drop하기

In [62]:
# 결측값 확인
train_x.isnull()

,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,True,True,True
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,True,True,True
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,True,True,True
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,True,True,True
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,True,True,True
594,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,True,True,True
595,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,True,True,True
596,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,True,True,True


In [63]:
train_x.isnull().sum()

LINE              0
PRODUCT_CODE      0
X_1               0
X_2               0
X_3               0
               ... 
X_2871            0
X_2872          598
X_2873          598
X_2874          598
X_2875          598
Length: 2877, dtype: int64

In [64]:
# 결측값있는 열 전체 삭제
train_x = train_x.dropna(axis =1)
test_x = test_x.dropna(axis=1)

In [65]:
train_x

,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,...,X_2862,X_2863,X_2864,X_2865,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871
0,2,0,0.013821,0.541547,0.0,0.0,0.39255,0.0,0.223664,0.048711,...,0.918723,0.100000,0.113966,0.119565,0.256757,0.248647,0.000000,0.122283,0.890487,0.0
1,3,0,0.013821,0.541547,0.0,0.0,0.39255,0.0,0.223664,0.048711,...,0.869376,0.100000,0.107939,0.119565,0.240754,0.300866,0.407899,0.164742,0.601770,0.0
2,2,0,0.013821,0.541547,0.0,0.0,0.39255,0.0,0.223664,0.048711,...,0.577649,0.100000,0.103476,0.119565,0.251422,0.133929,0.355835,0.205163,0.922566,0.0
3,3,0,0.013821,0.541547,0.0,0.0,0.39255,0.0,0.223664,0.048711,...,0.582003,0.110000,0.123562,0.119565,0.199858,0.202110,0.704129,0.003057,0.559181,0.0
4,2,0,0.013821,0.541547,0.0,0.0,0.39255,0.0,0.223664,0.048711,...,0.825835,0.100000,0.103810,0.108696,0.233997,0.275703,0.515978,0.088315,0.846239,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593,5,2,0.009804,0.533333,0.0,0.0,0.00000,0.0,0.294118,0.000000,...,0.545584,0.505582,0.520356,0.500698,0.664555,0.592741,0.612072,0.719083,0.275426,0.0
594,2,0,0.013821,0.541547,0.0,0.0,0.39255,0.0,0.223664,0.048711,...,0.624093,0.110000,0.130258,0.119565,0.616999,0.578193,0.658169,0.835938,0.266593,0.0
595,2,0,0.013821,0.541547,0.0,0.0,0.39255,0.0,0.223664,0.048711,...,0.448476,0.100000,0.100229,0.108696,0.664555,0.592741,0.612072,0.719083,0.275426,0.0
596,4,1,0.382353,0.466667,0.0,0.0,1.00000,0.0,0.000000,0.000000,...,0.545584,0.505582,0.520356,0.500698,0.664555,0.592741,0.612072,0.719083,0.275426,0.0


In [66]:
test_x

,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,...,X_2862,X_2863,X_2864,X_2865,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871
0,5,2,0.009804,0.466667,0.0,0.0,0.00000,0.0,0.352941,0.000000,...,0.545584,0.505582,0.520356,0.500698,0.664555,0.592741,0.612072,0.719083,0.275426,0.0
1,4,2,0.009804,0.400000,0.0,0.0,1.00000,0.0,0.000000,0.000000,...,0.545584,0.505582,0.520356,0.500698,0.664555,0.592741,0.612072,0.719083,0.275426,0.0
2,4,2,0.009804,0.533333,0.0,0.0,1.00000,0.0,0.000000,0.000000,...,0.545584,0.505582,0.520356,0.500698,0.664555,0.592741,0.612072,0.719083,0.275426,0.0
3,0,0,0.013821,0.541547,0.0,0.0,0.39255,0.0,0.223664,0.048711,...,0.843251,0.940000,0.930103,0.880435,0.664555,0.592741,0.612072,0.719083,0.275426,0.0
4,1,0,0.013821,0.541547,0.0,0.0,0.39255,0.0,0.223664,0.048711,...,0.783745,0.920000,0.919557,0.880435,0.664555,0.592741,0.612072,0.719083,0.275426,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305,5,2,0.009804,0.266667,0.0,0.0,0.00000,0.0,0.352941,0.000000,...,0.545584,0.505582,0.520356,0.500698,0.664555,0.592741,0.612072,0.719083,0.275426,0.0
306,4,2,0.009804,0.600000,0.0,0.0,1.00000,0.0,0.000000,0.000000,...,0.545584,0.505582,0.520356,0.500698,0.664555,0.592741,0.612072,0.719083,0.275426,0.0
307,5,2,0.009804,0.266667,0.0,0.0,0.00000,0.0,0.294118,0.000000,...,0.545584,0.505582,0.520356,0.500698,0.664555,0.592741,0.612072,0.719083,0.275426,0.0
308,5,2,0.009804,0.533333,0.0,0.0,0.00000,0.0,0.352941,0.000000,...,0.545584,0.505582,0.520356,0.500698,0.664555,0.592741,0.612072,0.719083,0.275426,0.0


## 5. Modeling

In [67]:
train_x.shape, train_y.shape, test_x.shape

((598, 2795), (598,), (310, 2795))

#### RandomForestClassifier

In [ ]:
start = time.time() # 시작
params = {
    'n_estimators':[500, 1000, 1500, 2000, 2500, 3300],
    'max_depth' : [5, 6, 8, 10, 12], 
    'min_samples_leaf' : [5, 6, 7, 8, 9],
    'min_samples_split' : [6, 7, 8, 9, 10, 12]
}
rf_cls = RandomForestClassifier(random_state = 37, n_jobs = 1)
grid_cv = GridSearchCV(rf_cls, param_grid = params, cv = 2, scoring = 'accuracy',n_jobs=1, verbose =2 )
grid_cv.fit(train_x, train_y)

# 최적의 파라미터 모델을 이용하여 예측값 생성
grid_estimator = grid_cv.best_estimator_
rf_pred = grid_estimator.predict(test_x)
# 후보 파라미터 성능 검증 결과 출력
print('최적 하이퍼 파라미터:\n', grid_cv.best_params_)
print('최고 예측 정확도: {0:.4f}'.format(grid_cv.best_score_))
sec = time.time()-start # 종료 - 시작 (걸린 시간)
times = str(datetime.timedelta(seconds=sec))
short = times.split(".")[0] # 초 단위 까지만
print(f"{times} sec")
print(f"{short} sec")

Fitting 2 folds for each of 900 candidates, totalling 1800 fits
[CV] END max_depth=5, min_samples_leaf=5, min_samples_split=6, n_estimators=500; total time=   1.0s
[CV] END max_depth=5, min_samples_leaf=5, min_samples_split=6, n_estimators=500; total time=   1.1s
[CV] END max_depth=5, min_samples_leaf=5, min_samples_split=6, n_estimators=1000; total time=   1.9s
[CV] END max_depth=5, min_samples_leaf=5, min_samples_split=6, n_estimators=1000; total time=   2.2s
[CV] END max_depth=5, min_samples_leaf=5, min_samples_split=6, n_estimators=1500; total time=   3.0s
[CV] END max_depth=5, min_samples_leaf=5, min_samples_split=6, n_estimators=1500; total time=   3.2s
[CV] END max_depth=5, min_samples_leaf=5, min_samples_split=6, n_estimators=2000; total time=   4.1s
[CV] END max_depth=5, min_samples_leaf=5, min_samples_split=6, n_estimators=2000; total time=   4.0s
[CV] END max_depth=5, min_samples_leaf=5, min_samples_split=6, n_estimators=2500; total time=   5.8s
[CV] END max_depth=5, min_sam

[CV] END max_depth=5, min_samples_leaf=6, min_samples_split=6, n_estimators=2500; total time=   4.9s
[CV] END max_depth=5, min_samples_leaf=6, min_samples_split=6, n_estimators=3300; total time=   6.6s
[CV] END max_depth=5, min_samples_leaf=6, min_samples_split=6, n_estimators=3300; total time=   7.4s
[CV] END max_depth=5, min_samples_leaf=6, min_samples_split=7, n_estimators=500; total time=   1.0s
[CV] END max_depth=5, min_samples_leaf=6, min_samples_split=7, n_estimators=500; total time=   1.0s
[CV] END max_depth=5, min_samples_leaf=6, min_samples_split=7, n_estimators=1000; total time=   2.0s
[CV] END max_depth=5, min_samples_leaf=6, min_samples_split=7, n_estimators=1000; total time=   1.9s
[CV] END max_depth=5, min_samples_leaf=6, min_samples_split=7, n_estimators=1500; total time=   2.9s
[CV] END max_depth=5, min_samples_leaf=6, min_samples_split=7, n_estimators=1500; total time=   3.1s
[CV] END max_depth=5, min_samples_leaf=6, min_samples_split=7, n_estimators=2000; total time=

[CV] END max_depth=5, min_samples_leaf=7, min_samples_split=7, n_estimators=2000; total time=   4.7s
[CV] END max_depth=5, min_samples_leaf=7, min_samples_split=7, n_estimators=2500; total time=   5.2s
[CV] END max_depth=5, min_samples_leaf=7, min_samples_split=7, n_estimators=2500; total time=   5.1s
[CV] END max_depth=5, min_samples_leaf=7, min_samples_split=7, n_estimators=3300; total time=   6.4s
[CV] END max_depth=5, min_samples_leaf=7, min_samples_split=7, n_estimators=3300; total time=   6.6s
[CV] END max_depth=5, min_samples_leaf=7, min_samples_split=8, n_estimators=500; total time=   0.9s
[CV] END max_depth=5, min_samples_leaf=7, min_samples_split=8, n_estimators=500; total time=   1.0s
[CV] END max_depth=5, min_samples_leaf=7, min_samples_split=8, n_estimators=1000; total time=   2.0s
[CV] END max_depth=5, min_samples_leaf=7, min_samples_split=8, n_estimators=1000; total time=   2.3s
[CV] END max_depth=5, min_samples_leaf=7, min_samples_split=8, n_estimators=1500; total time=

[CV] END max_depth=5, min_samples_leaf=8, min_samples_split=8, n_estimators=1500; total time=   2.9s
[CV] END max_depth=5, min_samples_leaf=8, min_samples_split=8, n_estimators=2000; total time=   3.8s
[CV] END max_depth=5, min_samples_leaf=8, min_samples_split=8, n_estimators=2000; total time=   4.0s
[CV] END max_depth=5, min_samples_leaf=8, min_samples_split=8, n_estimators=2500; total time=   4.8s
[CV] END max_depth=5, min_samples_leaf=8, min_samples_split=8, n_estimators=2500; total time=   5.0s
[CV] END max_depth=5, min_samples_leaf=8, min_samples_split=8, n_estimators=3300; total time=   6.3s
[CV] END max_depth=5, min_samples_leaf=8, min_samples_split=8, n_estimators=3300; total time=   6.6s
[CV] END max_depth=5, min_samples_leaf=8, min_samples_split=9, n_estimators=500; total time=   1.0s
[CV] END max_depth=5, min_samples_leaf=8, min_samples_split=9, n_estimators=500; total time=   1.0s
[CV] END max_depth=5, min_samples_leaf=8, min_samples_split=9, n_estimators=1000; total time=

[CV] END max_depth=5, min_samples_leaf=9, min_samples_split=9, n_estimators=1000; total time=   2.0s
[CV] END max_depth=5, min_samples_leaf=9, min_samples_split=9, n_estimators=1500; total time=   2.8s
[CV] END max_depth=5, min_samples_leaf=9, min_samples_split=9, n_estimators=1500; total time=   3.0s
[CV] END max_depth=5, min_samples_leaf=9, min_samples_split=9, n_estimators=2000; total time=   3.8s
[CV] END max_depth=5, min_samples_leaf=9, min_samples_split=9, n_estimators=2000; total time=   4.0s
[CV] END max_depth=5, min_samples_leaf=9, min_samples_split=9, n_estimators=2500; total time=   4.8s
[CV] END max_depth=5, min_samples_leaf=9, min_samples_split=9, n_estimators=2500; total time=   4.9s
[CV] END max_depth=5, min_samples_leaf=9, min_samples_split=9, n_estimators=3300; total time=   6.3s
[CV] END max_depth=5, min_samples_leaf=9, min_samples_split=9, n_estimators=3300; total time=   6.7s
[CV] END max_depth=5, min_samples_leaf=9, min_samples_split=10, n_estimators=500; total tim

...

[CV] END max_depth=10, min_samples_leaf=5, min_samples_split=12, n_estimators=500; total time=   1.2s
[CV] END max_depth=10, min_samples_leaf=5, min_samples_split=12, n_estimators=500; total time=   1.2s
[CV] END max_depth=10, min_samples_leaf=5, min_samples_split=12, n_estimators=1000; total time=   2.6s
[CV] END max_depth=10, min_samples_leaf=5, min_samples_split=12, n_estimators=1000; total time=   2.6s
[CV] END max_depth=10, min_samples_leaf=5, min_samples_split=12, n_estimators=1500; total time=   3.7s
[CV] END max_depth=10, min_samples_leaf=5, min_samples_split=12, n_estimators=1500; total time=   3.4s
[CV] END max_depth=10, min_samples_leaf=5, min_samples_split=12, n_estimators=2000; total time=   5.3s
[CV] END max_depth=10, min_samples_leaf=5, min_samples_split=12, n_estimators=2000; total time=   4.8s
[CV] END max_depth=10, min_samples_leaf=5, min_samples_split=12, n_estimators=2500; total time=   5.8s
[CV] END max_depth=10, min_samples_leaf=5, min_samples_split=12, n_estimato

[CV] END max_depth=10, min_samples_leaf=6, min_samples_split=12, n_estimators=2500; total time=   6.0s
[CV] END max_depth=10, min_samples_leaf=6, min_samples_split=12, n_estimators=3300; total time=   7.8s
[CV] END max_depth=10, min_samples_leaf=6, min_samples_split=12, n_estimators=3300; total time=   8.0s
[CV] END max_depth=10, min_samples_leaf=7, min_samples_split=6, n_estimators=500; total time=   1.0s
[CV] END max_depth=10, min_samples_leaf=7, min_samples_split=6, n_estimators=500; total time=   1.1s
[CV] END max_depth=10, min_samples_leaf=7, min_samples_split=6, n_estimators=1000; total time=   2.3s
[CV] END max_depth=10, min_samples_leaf=7, min_samples_split=6, n_estimators=1000; total time=   2.4s
[CV] END max_depth=10, min_samples_leaf=7, min_samples_split=6, n_estimators=1500; total time=   3.4s
[CV] END max_depth=10, min_samples_leaf=7, min_samples_split=6, n_estimators=1500; total time=   3.4s
[CV] END max_depth=10, min_samples_leaf=7, min_samples_split=6, n_estimators=2000

[CV] END max_depth=10, min_samples_leaf=8, min_samples_split=6, n_estimators=2000; total time=   4.3s
[CV] END max_depth=10, min_samples_leaf=8, min_samples_split=6, n_estimators=2000; total time=   4.5s
[CV] END max_depth=10, min_samples_leaf=8, min_samples_split=6, n_estimators=2500; total time=   5.5s
[CV] END max_depth=10, min_samples_leaf=8, min_samples_split=6, n_estimators=2500; total time=   5.4s
[CV] END max_depth=10, min_samples_leaf=8, min_samples_split=6, n_estimators=3300; total time=   7.3s
[CV] END max_depth=10, min_samples_leaf=8, min_samples_split=6, n_estimators=3300; total time=   7.5s
[CV] END max_depth=10, min_samples_leaf=8, min_samples_split=7, n_estimators=500; total time=   1.1s
[CV] END max_depth=10, min_samples_leaf=8, min_samples_split=7, n_estimators=500; total time=   1.0s
[CV] END max_depth=10, min_samples_leaf=8, min_samples_split=7, n_estimators=1000; total time=   2.1s
[CV] END max_depth=10, min_samples_leaf=8, min_samples_split=7, n_estimators=1000; t

[CV] END max_depth=10, min_samples_leaf=9, min_samples_split=7, n_estimators=1000; total time=   2.2s
[CV] END max_depth=10, min_samples_leaf=9, min_samples_split=7, n_estimators=1500; total time=   3.1s
[CV] END max_depth=10, min_samples_leaf=9, min_samples_split=7, n_estimators=1500; total time=   3.2s
[CV] END max_depth=10, min_samples_leaf=9, min_samples_split=7, n_estimators=2000; total time=   4.1s
[CV] END max_depth=10, min_samples_leaf=9, min_samples_split=7, n_estimators=2000; total time=   4.3s
[CV] END max_depth=10, min_samples_leaf=9, min_samples_split=7, n_estimators=2500; total time=   5.5s
[CV] END max_depth=10, min_samples_leaf=9, min_samples_split=7, n_estimators=2500; total time=   5.8s
[CV] END max_depth=10, min_samples_leaf=9, min_samples_split=7, n_estimators=3300; total time=   6.7s
[CV] END max_depth=10, min_samples_leaf=9, min_samples_split=7, n_estimators=3300; total time=   6.7s
[CV] END max_depth=10, min_samples_leaf=9, min_samples_split=8, n_estimators=500; 

[CV] END max_depth=12, min_samples_leaf=5, min_samples_split=8, n_estimators=500; total time=   1.1s
[CV] END max_depth=12, min_samples_leaf=5, min_samples_split=8, n_estimators=500; total time=   1.1s
[CV] END max_depth=12, min_samples_leaf=5, min_samples_split=8, n_estimators=1000; total time=   2.4s
[CV] END max_depth=12, min_samples_leaf=5, min_samples_split=8, n_estimators=1000; total time=   2.4s
[CV] END max_depth=12, min_samples_leaf=5, min_samples_split=8, n_estimators=1500; total time=   3.7s
[CV] END max_depth=12, min_samples_leaf=5, min_samples_split=8, n_estimators=1500; total time=   3.5s
[CV] END max_depth=12, min_samples_leaf=5, min_samples_split=8, n_estimators=2000; total time=   4.5s
[CV] END max_depth=12, min_samples_leaf=5, min_samples_split=8, n_estimators=2000; total time=   4.4s


- n_jobs : 모델 학습 및 예측에 사용되는 병렬 작업의 수 -1의 경우 cpu의 모든 코어 사용
- n_estimators : 생성할 트리의 개수(default = 100)
- min_samples_leaf : 노드를 분할하기 위해 리프 노드가 가져야 할 최소한의 샘플 개수 
  > 값을 작게 설정하면 모델 복잡성 증가:과적합 위험 값을 증가 시키면 모델의 편향은 증가하고 분산은 감소. 모델의 복잡도가 낮아지지만 일부 손실 발생
- min_samples_split : 노드를 분할 하기 위해 필요한 최소한의 샘플 개수
  > 샘플 수가 이보다 작아지면 분할하지 않고 리프 노드로 설정 // 값이 작을 수록 복잡성 증가. 데이터 작은 변동에 민감반응 과적합위험.
- max_features : 노드를 분할 할 때 고려 할 특성의 무작위 하위 집합의 크기
  > 값이 작으면 분산 감소, 편향증가, 복잡도 감소 , 모델 성능 감소
- max_depth = 트리의 최대 깊이(defualt = none) : 깊이가 제한된 트리는 각각의 결정 경계를 더 간단하게 만듬. 모델 과적합 방지
  > 값이 작으면 모델의 단순성을 증가. 분산감소 편향 증가 => 데이터의 다양한 패턴 잡아내지 못하고 일반화 능력 향상됨
- feaure_importances_ : 랜덤포레스트는 다수의 의사결정 트리 앙상블하여 모델 구성. 각 피처들을 사용하여 분할 수행하고 이 과정에서 피처들의 중요도 계산

- best_estimator : gridsearchCV 객체에서 최적의 추정기(estimator)를 반환하는 속성


#### OOF(Out Of Fold) prediction & LGBMClassifier

> 검증데이터 학습데이터 분리

In [71]:
cut = int(len(train_x)*0.8)
train_xx = train_x[:cut]# 앞에서부터 80퍼까지
valid_xx = train_x[cut:]# 80퍼이후 20퍼 데이터
train_yy = train_y[:cut]
valid_yy = train_y[cut:]
print(train_xx.shape, train_yy.shape, valid_xx.shape, valid_yy.shape)

(478, 2795) (478,) (120, 2795) (120,)


In [70]:
t_train_xx,t_valid_xx,t_train_yy,t_valid_yy = train_test_split(train_x,train_y,test_size = 0.2, shuffle = True, random_state=42)
print(t_train_xx.shape, t_train_yy.shape, t_valid_xx.shape, t_valid_yy.shape)

(478, 2795) (478,) (120, 2795) (120,)


In [74]:
t_valid_yy.unique()

array([0, 1, 2], dtype=int64)

> OPTUNA를 이용한 최적 하이퍼파라미터 찾기

In [ ]:
from optuna.samplers import TPESampler# objective와 study를 정의하고, n_trails 파라미터를 조정하여 몇 회의 trial 수행할지 설정하는 방식
sampler = TPESampler(seed = 10)
def objective(trial):
    dtrain = lgb.Dataset(t_train_xx, label = t_train_yy)
    dtest = lgb.Dataset(t_valid_xx, label = t_valid_yy)
    param = {
        'objective': 'multiclass', # 분류
        'verbose': -1, #학습 출력 결과 n_estimators 실행회수 100번마다 결과 출력
        'metric': 'multi_logloss', #평가지표
#       'metric': 'rmse', # 회귀모델 평가지표 
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 3e-5), #L1 Regularization parameter로 불필요한 가중치를 0으로 만든다.
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 9e-2),#L2 Regularization parameter로 불필요한 가중치를 0으로 만든다.
        "num_leaves": trial.suggest_int("num_leaves", 2, 256), #tree의 최대 leaf node의 개수이다.
        'max_depth': trial.suggest_int('max_depth',3, 15), #suggest_int 범위 내의 정수형 값 선택 // 깊이 제한 하는 파라미터 깊이제한이 클수록 과적합이 발생하는 것이
        'learning_rate': trial.suggest_loguniform("learning_rate", 1e-8, 1e-2), #suggest_loguniform 범위 내의 로그 함수 선상값 선택// 학습률을 의미
        'n_estimators': trial.suggest_int('n_estimators', 100, 3000), # 반복훈련 하려는 트리 개수 지정
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100), # 주요 하이퍼파라미터, 큰 값으로 설정하면 트리가 깊이지는것을 방지 // leaf node에 포함 되어야 할 최소한의 데이터 개수 의미
        'subsample': trial.suggest_loguniform('subsample', 0.4, 1),# boosting 단계마다 데이터를 랜덤으로 선택하는 비율. 즉 subsample 비율 만큼 데이터를 랜덤으로 선택해서 학습 진행// 값이 작으면 속도를 향상 할수 잇고 overfitting방지
        'colsample_bytree': trial.suggest_float("colsample_bytree", 0.4, 1.0),#boosting 단계마다 feature를 선택하는 비율이다. feature가 많거나 소수의 feature에 의존적일때 overfitting해결 할 수 있다.
    }
    
    model_lgb = lgb.LGBMClassifier(**param)## *arg는 튜플형식가변인자, **kwargs 딕셔너리 키워드 
    lgb_model = model_lgb.fit(t_train_xx, t_train_yy, eval_set =[(t_valid_xx, t_valid_yy)], verbose = False, early_stopping_rounds = 100)#eval_set 검증데이터 셋 지정, early_stoping_rounds : 검증데이터n개를 학습기를 통해 가중치 부여하며 오류개선 -> n_estimators횟수반복 ->loss나 성능지표가 25회 동안 향상되지 않으면 조기종료
    lgb_pred = lgb_model.predict_proba(t_valid_xx)
    #rmse = RMSE(valid_yy, lgb_pred) : 평균제곱오차 회귀 예측 모델로 예측값과 실제값 사이의 평균 측정
    log_score = log_loss(t_valid_yy,t_lgb_pred) # 분류문제의 대표적인 평가지표 교차 엔트로피라고 부르기도함. 실제 값을 예측하는 확률에 로그를 취하여 부호를 반전 시킨값. 낮을수록 좋은 지표
    return log_score
#study_name='lgbm_papameter_opt',
study_lgb = optuna.create_study(direction='minimize', sampler=sampler) #>> study 정의 minimize: logloss를 최소화 시켜주는 방향으로 튜닝하고 싶을경우. accuracy, roc-auc 같은 경우는 최대화 시켜주는 방향으로 maximize로 설정
study_lgb.optimize(objective, n_trials = 20) # optimize 최적함수 파라미터 찾는것. 즉 최적화

#과적합시 파라미터 튜닝
#min_chlid_weight 높이기, max_depth 낮추기. subsample, 트리 복잡도 낮추기, bagging_fraction[dafault = 1.0]: 트리가 커져서 과적합 하는것을 제어(==subsample) 샘플링 비율지정
trial = study_lgb.best_trial
trial_params = trial.params
print(f"best Trial:score{trial.value},\nparams{trial_params}")
print("Best Score:", study_lgb.best_value)
print("Best trial:", study_lgb.best_trial.params)

Hyperparameter optimization을 수행하기 위해 objective와 study를 정의하고, n_trials 파라미터를 조정하여 몇 회의 trial을 수행할지 설정할 수 있다.
- objective : 매 trial을 input으로 받는 함수
- Study : objective 함수에 기반하여 optimization을 수행하는 하나의 프로젝트
- Trial : Study 내의 optimization 단일 수행

> optuna를 활용한 최적의 하이퍼파라미터 시각화

In [ ]:
optuna.visualization.plot_optimization_history(study_lgb)

In [ ]:
#파라미터들관의 관계
optuna.visualization.plot_parallel_coordinate(study_lgb)

In [ ]:
# 각 파라미터들의 상관관계
optuna.visualization.plot_contour(
    study_lgb,
    params=[
        "reg_alpha",
        "reg_lambda",
        "num_leaves",
        'max_depth',
        'learning_rate',
        'n_estimators',
        'min_child_samples',
        'subsample',
        'colsample_bytree',
    ],
)

In [ ]:
# 하이퍼 파라미터 중요도
optuna.visualization.plot_param_importances(study_lgb)

> OOF(out of fold) prediction 4fold 학습

In [ ]:
#oof 4번학습 결과 도출
folds = KFold(n_splits = 4, shuffle = True, random_state = 37)
# validation set가 n_split갯수만큼 있으므로 크기는 ftr_app의 크기가 되어야 함. 
oof_preds = np.zeros((train_x.shape[0],3))# 학습된 모델의 valitation set 을 예측하여 결과 확률을 담을 array 생성// np.zeros : 0으로 채워진 [shape] 생성
# Ouf of Folds로 학습된 모델의 test dataset을 예측하여 결과 확률을 담을 array 생성. 
test_preds = np.zeros((test_x.shape[0],3))
# train 데이터를 8:2 비율로 분할하여 학습과 검증에 사용
# 마찬가지로 데이터가 섞이지 않게, 검증 데이터는 train 데이터의 마지막 20%을 사용
final_lgb_model = lgb.LGBMClassifier(**trial_params)#optuna로 찾은 최적 하이퍼파라미터 적용
for fold_idx, (train_idx, valid_idx) in enumerate(folds.split(train_x)):#train_x 데이터를 count idx, 학습 검증데이터 분류 idx
    print('##### interation', fold_idx, ' 시작')
    train_xxx = train_x.iloc[train_idx, :]#분류된 train_idx 값에 맞는 데이터 저장
    train_yyy = train_y.iloc[train_idx]
    valid_xxx = train_x.iloc[valid_idx, :]
    valid_yyy = train_y.iloc[valid_idx]
    final_lgb_model.fit(train_xxx, train_yyy, eval_set=[(train_xxx, train_yyy), (valid_xxx, valid_yyy)], verbose=200, early_stopping_rounds = 200)
    oof_preds[valid_idx] = final_lgb_model.predict_proba(valid_xxx, num_iteration=final_lgb_model.best_iteration_)
    test_preds += final_lgb_model.predict_proba(test_x, num_iteration = final_lgb_model.best_iteration_)/folds.n_splits

In [ ]:
test_preds

> 예측된 0 1 2인덱스에 대한 최대값 추출

In [ ]:
lgbm_preds= []
for i in range(len(test_preds)):
     lgbm_preds.append(np.argmax(test_preds[i]))
lgbm_pred = np.array(lgbm_pred)
print(lgbm_preds)

#### ExtraTreesClassifier

> 분류를 위한 앙상블 학습 알고리즘 중 하나로, 무작위 디시전트리를 사용하는 방법이다.

-  랜덤포레스트와 유사하지만 극도로 무작위화된 특징을 가진다. 
- random threshold를 사용하여 더욱 랜덤한 트리를 생성하기 때문에 모델의 분산을 줄이고 오버피팅을 방지 할 수 있다.
- 극도로 무작위화(Extremely Randomized)된 기계 학습 방법
- 데이터 샘플 수와 특성 설정까지 랜덤
- 랜덤 포레스트와 동일한 원리를 이용하기 때문에 많은 특성을 공유함
- 랜덤 포레스트에 비해 속도가 빠르고 성능도 미세하게 높음
- Bootstrap 샘플링을 사용하지 않고 전체 특성 중 일부를 랜덤하게 선택해 노드 분할에 사용 → 무작위 분할 중 가장 좋은 것을 분할 규칙으로 선택

> ExtraTreesClassifier 모델 생성

In [ ]:
ex_cls = ExtraTreesClassifier(random_state=37)

> 교차검증사용 // 교차검증을 사용하면 모델의 예측 성능을 미리 예측하고, 과적합 문제를 방지 할 수 있다

- cross_val_score : 각 검증의 평가 스코어 도출

In [ ]:
valid_ex_preds = cross_val_score(ex_cls, train_x, train_y,scoring='accuracy', cv =5)
print(f"각 검증 별 점수 : \n{pd.Series(valid_ex_preds)}\n")
print(f"평균 점수 : {valid_ex_preds.mean()}\n")

- cross_validate : 각 검증의 결과 도출

In [ ]:
score_ex_preds = cross_validate(ex_cls, train_x, train_y, scoring=['accuracy'], cv= 5,return_train_score=True)
print("각 검증 결과 : \n", pd.DataFrame(score_ex_preds))
print(score_ex_preds['test_accuracy'].mean())

> ExtraTreesClassifier 모델학습

In [ ]:
ex_cls.fit(train_x,train_y)
ex_preds = ex_cls.predict(test_x)

> 모델 피처의 중요도

In [ ]:
fi = pd.Series(ex_cls.feature_importances_)
fi.index = train_x.columns
fi.sort_values(ascending=False).head(10) # 내림차순 정렬

#### 간접 투표 분류기(Soft Voting) Predict

> 간접 투표(soft voting)는 모든 분류모델의 결과를 고려하는 투표 과정이다.
> 클래스 레이블에 대한 예측 확률을 합하고 확률이 가장 

> soft voting 할 모델 생성 및 하이퍼파라미터 입력

In [ ]:
trial_params = {'random_state' : 37,'reg_alpha': 8.597475168789943e-06, 'reg_lambda': 0.08528190763167148, 'num_leaves': 193, 'max_depth': 9, 'learning_rate': 0.007446401724927197, 'n_estimators': 158, 'min_child_samples': 63, 'subsample': 0.9980338359481813, 'colsample_bytree': 0.9687581255197978}
best_rf_params =  {'random_state' : 37,'n_jobs' : 1 ,'max_depth': 5, 'min_samples_leaf': 7, 'min_samples_split': 6, 'n_estimators': 2000}
sv_ex_cls = ExtraTreesClassifier(random_state=37)
sv_ex_cls.fit(train_x,train_y)
sv_lgb_model = lgb.LGBMClassifier(**trial_params)
sv_lgb_model.fit(train_x,train_y)
sv_rf_cls = RandomForestClassifier(**best_rf_params)
sv_rf_cls.fit(train_x,train_y)

> soft voting model 생성 및 학습 예측

In [ ]:
voting_clf = VotingClassifier([
    ("lgb_clf",sv_lgb_model),
    ("rf_clf",sv_rf_cls),
    ("ex_clf",sv_ex_cls)
], voting= 'soft')
voting_clf.fit(train_x,train_y)
voting_pred = voting_clf.predict(test_x)

## Submit

In [ ]:
submit = pd.read_csv('../sample_submission.csv')

- RandomForestClassifier_pred

In [ ]:
submit['Y_Class'] = rf_pred
submit.to_csv('./rf_pred.csv',index=False)

- LightGBMClassifier_pred

In [ ]:
submit['Y_Class'] = lgbm_pred
submit.to_csv('./lgbm_pred.csv',index=False)

- ExtraTreesClassifier_pred

In [ ]:
submit['Y_Class'] = ex_preds
submit.to_csv('./ex_preds.csv',index=False)

- Soft voting_pred

In [ ]:
submit['Y_Class'] = voting_pred
submit.to_csv('./voting_pred.csv',index=False)